In [ ]:
# Impotar as bibliotecas
import win32com.client as win32
import pandas as pd
import locale
import os
import numpy as np
import fnmatch
locale.setlocale(locale.LC_ALL, 'pt_BR')

In [46]:
# construir instância do aplicativo Outlook
#olNS = olApp.GetNAmeSpace('MAPI')
outlook = win32.Dispatch('outlook.application')
mailItem = outlook.CreateItem(0)

In [47]:
# Importar base de dados
df = pd.read_excel('Boletos/Pt2 - Relação de boletos emitidos para aposentados 072023.xls')

In [48]:
# Suponha que você tenha uma lista de arquivos com o CPF no nome
arquivos = ['Boletos/Pt2 - Relação de boletos emitidos para aposentados 072023.xls']

In [49]:
# Obtém o caminho absoluto para o diretório atual
diretorio_atual = os.getcwd()

# Obtém o caminho absoluto para a imagem de assinatura
caminho_imagem_assinatura = os.path.join(diretorio_atual, "Assinatura.jpg")

# Define a variável de assinatura com a tag <img> correta
assinatura = f'<p>Atenciosamente,</p><img src="{caminho_imagem_assinatura}" alt="Assinatura">'

In [ ]:
# verificando relacao
df.head(5)

In [ ]:
# verificando relacao
df.tail(5)

In [ ]:
#Find Shape
df.shape

In [ ]:
df.columns

In [ ]:
#Find Data Types
df.dtypes

In [ ]:
df.info

In [ ]:
diretorio_atual = os.getcwd()
assinatura = f'<p>Atenciosamente,</p><img src="cid:assinatura" alt="Assinatura">'

for index, row in df.iterrows():
    enderecos = row["E-MAIL"]
    nome_coop = row['Nome']
    valor = row['Valor']
    numero_formatado = '{:,.2f}'.format(valor).replace(',', 'X').replace('.', ',').replace('X', '.')
    nosso_numero = row["Nosso Número"]
    boleto = str(nosso_numero)
    nome_arquivo = row['Nosso Número']
        
    arquivo = os.path.join(os.getcwd(), 'boletos', f'{nosso_numero}.pdf')
    
    email = outlook.CreateItem(0)
    email.To = enderecos
    email.Subject = f'VENCIMENTO DE BOLETO COOPERATIVA - {nome_coop}'
    email.HTMLBody = email.HTMLBody.replace("cid:assinatura", f"<img src=\"{caminho_imagem_assinatura}\" alt=\"Assinatura\">")
     
    email.HTMLBody = f"""
        <p><html><body>Prezado (a) Cooperado (a) <b>{nome_coop}</b>.</p>
    
        <p> Informamos que o seu boleto no valor de <b>R$ {numero_formatado}</b> referente a cota de capital e/ou parcela de empréstimo vencerá em 31/07/2023.</p>

        <p> O boleto foi encaminhado por correio, está disponível no DDA do seu Banco e também em anexo.</p>

        <p> Para mais informações e demais dúvidas favor entrar em contato com a equipe de cobrança através do e-mail <b>cobranca.cooperativa@sescsp.org.br</b>.</p>
    
        <p> Caso já tenha efetivado o pagamento desconsiderar este e-mail.</p>

        <p> Atenciosamente,</p>
    
        <img src="cid:assinatura" alt="Assinatura">
    
        """
        
    assinatura_id = "Assinatura.jpg"
    email.Attachments.Add(caminho_imagem_assinatura, 1, 0, "Assinatura.jpg")
    email.Attachments.Add(arquivo) #arquivo do pdf
    email.HTMLBody = email.HTMLBody.replace("cid:assinatura", f"cid:{assinatura_id}")
    
    email.Send()
    
print('E-MAILS ENVIADOS')